In [ ]:
from pymongo import MongoClient
import bs4 as bs
import urllib.request
import requests
import csv
import re
import pandas as pd
import numpy as np
import time
import json
import sys
from datetime import datetime

#connect to MongoDB using pymongo
client = MongoClient('localhost', 27017)
db = client['13F']
collection = db['13F_collection']

def main():
    #open the csv containing CIK number for reading
    with open('/Users/a/Desktop/KSU/GRA/CSV/13f/13f test.csv' , newline= '') as csv_file:
        cik_reader = csv.reader(csv_file)
        diction_lists = []
        #check_point will keep track of what iteration the script is on by printing 1.. 2.. 3.. etc..
        check_point = 0
        for x in cik_reader:
            check_point = check_point + 1
            print(check_point, x)
            cik = ''.join(x)
            url_func(cik)

#Url where form 4 data will be retrieved
def url_func(cik_num):
    
    #insert variable will be appended to the end of the URL. Insert variable integer will display number of links
    insert = 0
    for b in range(10):
        
        #this will return the page that links to where CIK documents are stored. The part of url that reads 'page=' will set the amount of form 4's. Max # is 100
        url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + cik_num + '&type=13f&dateb=&owner=include&start=' + str(insert) + '&count=100'
        insert = insert + 100
        print(url)
        
        #This section spoofs a user agent to prevent blocking of scripts
        time.sleep(.5)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        spoof = requests.get(url, headers=headers)
        source = spoof.text
        
        #run beautifulSoup on returned data
        source_bs = bs.BeautifulSoup(source)

        #parse data to find all tags with 'a' (which are links) and an id of 'documentsbutton'
        #these links will need to have "https://www.sec.gov" appended to the front of them in next step
        document_id = source_bs.find_all("a", id="documentsbutton")
        
        #this loop will append "https://www.sec.gov" to the front of the links in 'document_id'
        #link_list[] will then contain the full links needed to proceed to the page where xml is stored. 
        link_list = []
        for x in document_id:
            if x.has_attr('href'):
                orange = x['href']
                link_list.append("https://www.sec.gov" + orange)

        #run each link in 'link_list[]' through urllib, then beautifulSoup, and extract 
        #all data from the table using 'tr'. This will provide links for the form4 XML document
        #store each iteration in 'xml_list'
        xml_list = []
        for link in link_list:
            time.sleep(.5)
            xml_data = urllib.request.urlopen(link).read()
            form4_bs = bs.BeautifulSoup(xml_data)
            form4_condensed = form4_bs.find_all('tr')       
            xml_list.extend(form4_condensed)
        
        #extract only the link containing the XML document from each iteration of new_xml_list
        #use 'text= re.compile(r"\.xml$"' to find link anchor text that ends in .xml
        #each .xml link will be an index in href_list[]
        href_list = []
        for x in xml_list:
            xml_link = x.findAll('a', href=True, text=re.compile(r"form13fInfoTable.xml$"))    
            href_list.extend(xml_link)


        #remove tags and append "https://www.sec.gov" to links in href_list[] and 
        #store in new list new_href_list[]
        new_href_list = []
        for x in href_list:
            if x.has_attr('href'):
                purple = x['href']
                new_href_list.append("https://www.sec.gov" + purple) 

        #create a dictionary with each key being the column name for the values
        diction_13f = {'nameOfIssuer': ([]),
                        'titleOfClass': ([]),
                        'cusip': ([]),
                        'value': ([]),
                        'sshPrnamt': ([]),
                        'sshPrnamtType': ([]),
                        'investmentDiscretion': ([]),
                        'otherManager': ([]),
                        'Sole': ([]),
                        'Shared': ([]),
                        'None': ([])}
        
        #create a dataframe for every .xml document link stored in new_href_list[]
        x_xml_links = []
        for link in new_href_list:
            print(link)
            time.sleep(.5)
            x_data = urllib.request.urlopen(link).read()
            source_bs = bs.BeautifulSoup(x_data, 'xml')
        
            #Each index of list_info will contain tags and text between infotable opening and closing tag
            list_info = []
            for a in source_bs.find_all('infoTable'):
                list_info.append(a)
              

            #iterate through beautifulSoup content in list_info, searching for tags with key value names found in diction_13f, 
            #append this information to diction_13f
            for x in list_info:
                for key in diction_13f:
                    diction_append = x.find(str(key))
                    if diction_append:
                        word_text = diction_append.text
                    else:
                        word_text = 'N/A'
                    diction_13f[key].append(word_text)
                    
        
        #convert diction_13f to a dataframe. If the dataframe is empty then pass, else output to json file. 
        df = pd.DataFrame(diction_13f)
        if df.empty:
            pass
        else:
            print(df)
            #output dictionary to json
            with open('/Users/a/Desktop/KSU/GRA/CSV/13f/13f.json', 'a') as json_text:
                json.dump(diction_13f, json_text)
            print("output complete")

main()